### O-AUTH FOR GEMINI FINE TUNING ACCESS

In [ ]:
import os
if 'COLAB_RELEASE_TAG' in os.environ:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
else:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'


You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=822844456417-7vgb06botm1nlqobgthdujc1k2vpu5ev.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=1w2zwSD6u4xa7YNm1bG2mdwGCqVeBn&access_type=offline&code_challenge=j3rYi6uxwNbXT6vBDNxVKoab8MD7YpBk0e3xhwGWr0I&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=1w2zwSD6u4xa7YNm1bG2mdwGCqVeBn&code=4/0ATx3LY4JBD-ZscOAvUU0WphE6_uSnh_j-O6XmvLD6qUgG_IMUo_9ZngM62EkuV2Z9RKTmQ&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generati

In [ ]:
import google.generativeai as genai


In [ ]:
for i, m in zip(range(10), genai.list_tuned_models()):
  print(m.name)


tunedModels/model3mapping-49pyrfyvaup8
tunedModels/model4mapping-4vkuhe9jeewz
tunedModels/model8mapping-kv5fvoq2gwrr
tunedModels/model9mmapping-ob9f8cxg2c7w
tunedModels/featuresoutputfinalcleaned-k74pmtufe9q9
tunedModels/featuresoutputfinalcleaned-anm89rdailys
tunedModels/middlepartgenerator-wf3x430emirw
tunedModels/classifierpeci-6uze6p5249yl
tunedModels/classifierpecimodel-lqys2wyq40t7


In [ ]:
model = genai.GenerativeModel(model_name='tunedModels/middlepartgenerator-wf3x430emirw')

In [ ]:
import numpy as np
import pandas as pd
df = pd.read_excel('check2.xlsx')

In [ ]:
df.tail()

,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order
7160,7160,SA24M005,FAC (Final Acceptance Certificate),NaN,0,Project Portion- II (Construction of 380kV ...,3
7161,7161,SA24M005,Remote Normalization Before Energization,NaN,1,-1,0
7162,7162,SA24M005,Normalization Before Energization EHV,NaN,0,Remote Normalization Before Energization No...,1
7163,7163,SA24M005,Energization for EHV,NaN,0,Remote Normalization Before Energization En...,2
7164,7164,SA24M005,Post Energization In Service Test for EHV,NaN,0,Remote Normalization Before Energization Po...,3


### Model usage with error handling for Middle Part of Code

In [ ]:
import time
import random
import string
from datetime import datetime
from google.api_core.exceptions import InternalServerError

predicted_cat = []
cnt = -1
cnt_req = 0
name = ''

# Total number of requests you want to process
total_requests = len(df['Concat'])

def generate_random_code(length=4):
    letters = string.ascii_letters
    return '#' + ''.join(random.choice(letters) for _ in range(length))

# Function to process requests with delay and retry mechanism
def process_requests(rate_limit, interval, max_retries=3, retry_delay=5):
    global cnt, cnt_req, name
    for i in df['Concat']:
        cnt += 1

        if i != '-1' and df['Order'][cnt] == 1:
            cnt_req += 1
            retries = 0
            while retries < max_retries:
                try:
                    if cnt_req % rate_limit == 0 and cnt_req != 0:
                        print(f"Sleeping .. {datetime.now()}:")
                        print(cnt_req)
                        time.sleep(interval)

                    result = model.generate_content(i)
                    name = result.text
                    break
                except ValueError:
                    print(f"Error processing request {cnt_req} and row number {cnt}. Generating random code.")
                    name = generate_random_code()
                    break
                except InternalServerError:
                    print(f"InternalServerError for request {cnt}. Retrying in {retry_delay} seconds.")
                    time.sleep(retry_delay)
                    retries += 1
                    if retries == max_retries:
                        print(f"Max retries reached for request {cnt}. Generating random code.")
                        name = generate_random_code()
            predicted_cat.append(name)
            print(f"Processed request {cnt} at {datetime.now()}")
            print(name, i)

        elif i != '-1' and (cnt == 7164 or df['Order'][cnt + 1] == 0):
            for _ in range(df['Order'][cnt] - 1):
                predicted_cat.append(name)

        if i == '-1':
            predicted_cat.append(' ')

# Rate limit of 16 requests per minute
rate_limit = 16
# Sleep interval of 61 seconds (1 minute and 1 second)
interval = 61

process_requests(rate_limit, interval)


Processed request 2 at 2024-06-26 10:28:57.872771
CM00MW    Contractual Milestones      Portion- I Complete Mobilization and Commence the work- Portion I
Processed request 7 at 2024-06-26 10:28:58.566933
CMOBW    Contractual Milestones      Portion- II Complete Mobilization and Commence the work- Portion II
Processed request 12 at 2024-06-26 10:28:59.187029
IMILC    Internal Milestones Contract Award Date
Error processing request 4 and row number 21. Generating random code.
Processed request 21 at 2024-06-26 10:28:59.845202
#fSZX    Project Portion- I (Construction of North Qassim 380kV BSP & Installation of New Reactors      Main Station- North Qassim 380kV BSP (9061 BSP)        Preliminary Works          General Works Kick off Meeting with Project Team
Processed request 37 at 2024-06-26 10:29:00.524937
PMOBK    Project Portion- I (Construction of North Qassim 380kV BSP & Installation of New Reactors      Main Station- North Qassim 380kV BSP (9061 BSP)        Preliminary Works        

### Saving Part of Code Generation

In [ ]:
df['Middle Part'] = predicted_cat
df.to_excel('Generated.xlsx')

In [ ]:
from google.colab import files
files.download('Generated.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Concating all Partial Codes

In [ ]:
df2 = pd.read_excel('Generated.xlsx')

In [ ]:
df2.head()

,Unnamed: 0.1,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order,Middle Part,Classification
0,0,0,SA24M005,Contractual Milestones,NaN,1,-1,0,,
1,1,1,SA24M005,Portion- I,NaN,2,-1,0,,
2,2,2,SA24M005,Complete Mobilization and Commence the work- P...,NaN,0,Contractual Milestones Portion- I Comp...,1,CM00MW,Category: Contracts
3,3,3,SA24M005,Technical Completion of the Project ( TCC )- P...,NaN,0,Contractual Milestones Portion- I Tech...,2,CM00MW,Category: Contracts
4,4,4,SA24M005,Preliminary Acceptance of the Project ( PAC )-...,NaN,0,Contractual Milestones Portion- I Prel...,3,CM00MW,Category: Contracts


In [ ]:
counter = -1
weird = []
for i in df2['Classification']:
  counter += 1
  if df2['Concat'][counter] != '-1':
    if 'Contracts' in i:
      df2['Classification'][counter] = 'L000'
    if 'Construction' in i:
      df2['Classification'][counter] = 'C000'
    elif 'Engineering' in i:
      df2['Classification'][counter] = 'E000'
    elif 'Procurement' in i:
      df2['Classification'][counter] = 'P000'
    elif 'Inspection' in i:
      df2['Classification'][counter] = 'I000'
    elif 'Commissioning' in i:
      df2['Classification'][counter] = 'M000'

    else:
      weird.append(counter)


Streaming output truncated to the last 5000 lines.
<ipython-input-313-ffc912ce25f1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Classification'][counter] = 'E000'
<ipython-input-313-ffc912ce25f1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Classification'][counter] = 'E000'
<ipython-input-313-ffc912ce25f1>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Classification'][counter] = 'E000'
<ipython-input-313-

In [ ]:
weird

[7093, 7094, 7095]

In [ ]:
df2[200:300]

,Unnamed: 0.1,Unnamed: 0,Job Code,DESCRIPTION,CODE,Hierarchy,Concat,Order,Middle Part,Classification
200,200,200,SA24M005,Submission,NaN,0,Project Portion- I (Construction of North Q...,2,CTSR,E000
201,201,201,SA24M005,Review / Evaluation,NaN,0,Project Portion- I (Construction of North Q...,3,CTSR,E000
202,202,202,SA24M005,Approval with Comments,NaN,0,Project Portion- I (Construction of North Q...,4,CTSR,E000
203,203,203,SA24M005,FINAL Approval,NaN,0,Project Portion- I (Construction of North Q...,5,CTSR,E000
204,204,204,SA24M005,Site Grading Plan & Sections,NaN,6,-1,0,,
...,...,...,...,...,...,...,...,...,...,...
295,295,295,SA24M005,Review / Evaluation,NaN,0,Project Portion- I (Construction of North Q...,3,E0GIS,E000
296,296,296,SA24M005,Approval with Comments,NaN,0,Project Portion- I (Construction of North Q...,4,E0GIS,E000
297,297,297,SA24M005,FINAL Approval,NaN,0,Project Portion- I (Construction of North Q...,5,E0GIS,E000
298,298,298,SA24M005,EHV GIS- VT (PT) Sizing Calculation,NaN,6,-1,0,,


In [ ]:
lst2 = []
cnt = -1
for j in df2['Order']:
    cnt+=1
    if j >=1 and j <10:
        lst2.append("00"+df2['Classification'][cnt][0]+str(j))
    elif j >=10 and j<100:
        lst2.append("0"+df2['Classification'][cnt][0]+str(j))
    else:
      lst2.append('X')

In [ ]:
len(df2)

7165

In [ ]:
df2['Last Four'] = lst2

In [ ]:
final_code = []

for i in range(len(df2)):
  if df2['Order'][i] !=0:
    x = str(df2['Classification'][i])+str(df2['Middle Part'][i])+ str(df2['Last Four'][i])
    final_code.append(str(x))
  else:
    final_code.append(' ')

In [ ]:
df2['Code Final'] = final_code

In [ ]:
final_code

[' ',
 ' ',
 'L000CM00MW00L1',
 'L000CM00MW00L2',
 'L000CM00MW00L3',
 'L000CM00MW00L4',
 ' ',
 'L000CMOBW00L1',
 'L000CMOBW00L2',
 'L000CMOBW00L3',
 'L000CMOBW00L4',
 ' ',
 'L000IMILC00L1',
 'L000IMILC00L2',
 'L000IMILC00L3',
 'L000IMILC00L4',
 'L000IMILC00L5',
 ' ',
 ' ',
 ' ',
 ' ',
 'E000#fSZX00E1',
 'E000#fSZX00E2',
 'E000#fSZX00E3',
 'E000#fSZX00E4',
 'E000#fSZX00E5',
 'E000#fSZX00E6',
 'E000#fSZX00E7',
 'E000#fSZX00E8',
 'E000#fSZX00E9',
 'E000#fSZX0E10',
 'E000#fSZX0E11',
 'E000#fSZX0E12',
 'E000#fSZX0E13',
 'E000#fSZX0E14',
 'E000#fSZX0E15',
 ' ',
 'E000PMOBK00E1',
 'E000PMOBK00E2',
 'E000PMOBK00E3',
 'E000PMOBK00E4',
 'E000PMOBK00E5',
 'E000PMOBK00E6',
 'E000PMOBK00E7',
 'E000PMOBK00E8',
 'E000PMOBK00E9',
 'E000PMOBK0E10',
 'E000PMOBK0E11',
 'E000PMOBK0E12',
 'E000PMOBK0E13',
 'E000PMOBK0E14',
 ' ',
 'E000PCW00E1',
 'E000PCW00E2',
 'E000PCW00E3',
 'E000PCW00E4',
 'E000PCW00E5',
 'E000PCW00E6',
 'E000PCW00E7',
 'E000PCW00E8',
 'E000PCW00E9',
 'E000PCW0E10',
 ' ',
 ' ',
 ' ',
 '

In [ ]:
df2.to_excel('Generated-final.xlsx')
from google.colab import files
files.download('Generated-final.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>